# Gather the data needed to train the model

In this notebook we combine all of the data from
step 2. The contents of this notebook is mirrored
in `utils.py` so that it can be used in other notebooks.

In [1]:
%load_ext jupyter_black

In [6]:
from pathlib import Path
import os

import pandas as pd

In [9]:
input_dir = Path(os.getenv("STORM_DATA_DIR")) / "analysis/02_new_model_input"

In [34]:
# Read in the building damage data. Drop any
# rows that have NA for the typhoon name.
filename = (
    input_dir
    / "02_housing_damage/output/percentage_building_damage_bygrid.csv"
)
df_damage = pd.read_csv(filename).dropna(subset="typhoon")

In [35]:
df_damage.columns

Index(['id', 'NUMPOINTS', 'left', 'top', 'right', 'bottom', 'Area', 'AreainKM',
       'Len', 'Longitude', 'Latitude', 'Centroid_x', 'Centroid_y',
       'ADM3_PCODE', 'ADM3_EN', 'Grid Completeness', 'Id', 'pcode', 'typhoon',
       'Year', 'Totally', 'Partially', 'total', 'Totally_Damaged_bygrid',
       'Partially_Damaged_bygrid', 'All_Damaged_bygrid',
       'Totally_Damaged_Perc_bygrid', 'Partially_Damaged_Perc_bygrid',
       'All_Damaged_Perc_bygrid'],
      dtype='object')

In [40]:
columns_to_keep = [
    "id",
    "pcode",
    "typhoon",
    "Year",
    "Totally",
    "Totally_Damaged_bygrid",
]
df_damage2 = df_damage.loc[:, columns_to_keep]
df_damage2

,id,pcode,typhoon,Year,Totally,Totally_Damaged_bygrid
6116,6097,PH175324000,LINFA,2015.0,0.0,0.000000
6120,6098,PH175324000,LINFA,2015.0,0.0,0.000000
6285,6262,PH175324000,LINFA,2015.0,0.0,0.000000
6287,6263,PH175324000,LINFA,2015.0,0.0,0.000000
6289,6264,PH175324000,LINFA,2015.0,0.0,0.000000
...,...,...,...,...,...,...
48370,20557,PH112508000,Lingling,2014.0,3.0,0.179991
48371,20557,PH112508000,Haiyan,2013.0,3.0,0.179991
48372,20558,PH112508000,Bopha,2012.0,80.0,6.997777
48373,20558,PH112508000,Lingling,2014.0,3.0,0.262417
